# Пример: Модель Стабильной Динамики и модель Бекмана

In [36]:
import pandas as pd
import numpy as np
import data_handler as dh
import model as md
import time
import matplotlib.pyplot as plt
from matplotlib import rc
import pickle

In [37]:
import numba
numba.__version__

'0.59.1'

In [38]:
sd_save = beckmann_save = './'
cities_data = 'cities_data/'
net_name = cities_data + 'Example_net_learn.tntp'
trips_name = cities_data + 'Example_trips_learn.tntp'

![title](github_example_pic.png)

- You can find the example's description at the following link:

https://doi.org/10.3390/math9111217

- There were more cases discussed in the following paper:

Nesterov, Y., de Palma, A. Stationary Dynamic Solutions in Congested Transportation Networks: Summary and Perspectives. Networks and Spatial Economics 3, 371–395 (2003). https://doi.org/10.1023/A:1025350419398

# Модель Бекмана

In [39]:
handler = dh.DataHandler()
graph_data = handler.GetGraphData(net_name, columns = ['init_node', 'term_node', 'capacity', 'free_flow_time'])
graph_correspondences, total_od_flow = handler.GetGraphCorrespondences(trips_name)

model = md.Model(graph_data, graph_correspondences, 
                    total_od_flow, mu = 0.25, rho = 0.15)

graph_data['graph_table'].head()

,init_node,init_node_thru,term_node,term_node_thru,capacity,free_flow_time
0,1,True,3,True,2000.0,1.0
1,1,True,3,True,2000.0,0.5
2,2,True,1,True,2000.0,0.5
3,2,True,1,True,2000.0,1.0
4,3,True,2,True,2000.0,1.0


In [40]:
graph_correspondences

{1: {'targets': [1, 2], 'corrs': [0.0, 2000.0]},
 2: {'targets': [2, 1], 'corrs': [0.0, 3000.0]}}

## Метод Франка-Вульфа

In [41]:
assert(model.mu == 0.25)
max_iter = 1000

print('Frank-Wolfe without stopping criteria')
solver_kwargs = {'max_iter' : max_iter, 'stop_crit': 'max_iter',
                 'verbose' : True, 'verbose_step': 200, 'save_history' : True}
tic = time.time()
result = model.find_equilibrium(solver_name = 'fwm', solver_kwargs = solver_kwargs)
toc = time.time()
print('Elapsed time: {:.0f} sec'.format(toc - tic))
print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')

Frank-Wolfe without stopping criteria
Oracles created...
Frank-Wolfe method...
Primal_init = 3852.63
Dual_init = -3500
Duality_gap_init = 352.627

Iterations number: 200
Primal_func_value = 3852.63
Dual_func_value = -3852.63
Duality_gap = 0
Duality_gap / Duality_gap_init = 0

Iterations number: 400
Primal_func_value = 3852.63
Dual_func_value = -3852.63
Duality_gap = 3.63798e-12
Duality_gap / Duality_gap_init = 1.03168e-14

Iterations number: 600
Primal_func_value = 3852.63
Dual_func_value = -3852.63
Duality_gap = 3.63798e-12
Duality_gap / Duality_gap_init = 1.03168e-14

Iterations number: 800
Primal_func_value = 3852.63
Dual_func_value = -3852.63
Duality_gap = -1.81899e-12
Duality_gap / Duality_gap_init = -5.15839e-15

Iterations number: 1000
Primal_func_value = 3852.63
Dual_func_value = -3852.63
Duality_gap = -1.36424e-12
Duality_gap / Duality_gap_init = -3.86879e-15

Result: success
Total iters: 1000
Primal_func_value = 3852.63
Dual_func_value = -3852.63
Duality_gap = -1.36424e-12
Du

In [42]:
print("times:", result['times'])
print("flows:", result['flows'])

times: [1.         0.575      0.8796875  1.         1.         0.73703704]
flows: [  -0. 2000. 3000.   -0.   -0. 2000.]


# Модель Стабильной Динамики

parameter $\mu = 0$

In [43]:
handler = dh.DataHandler()
graph_correspondences, total_od_flow = handler.GetGraphCorrespondences(trips_name)
graph_data = handler.GetGraphData(net_name, columns = ['init_node', 'term_node', 'capacity', 'free_flow_time'])
init_capacities = np.copy(graph_data['graph_table']['capacity'])
graph_data['graph_table'].head()

,init_node,init_node_thru,term_node,term_node_thru,capacity,free_flow_time
0,1,True,3,True,2000.0,1.0
1,1,True,3,True,2000.0,0.5
2,2,True,1,True,2000.0,0.5
3,2,True,1,True,2000.0,1.0
4,3,True,2,True,2000.0,1.0


In [44]:
graph_correspondences

{1: {'targets': [1, 2], 'corrs': [0.0, 2000.0]},
 2: {'targets': [2, 1], 'corrs': [0.0, 3000.0]}}

## Step 1: Base flows
Прежде всего, нам необходимо найти допустимое множество потоков на транспортном графе. Это необходимо для определения разрыва двойственности.

In [45]:
import numba

In [46]:
# start from 0.5,  0.75, 0.875 (according to our flows reconstruction method) 
alpha = 0.75
graph_data['graph_table']['capacity'] = init_capacities * alpha
model = md.Model(graph_data, graph_correspondences,
                 total_od_flow, mu = 0)

graph_data['graph_table'].head()

,init_node,init_node_thru,term_node,term_node_thru,capacity,free_flow_time
0,1,True,3,True,1500.0,1.0
1,1,True,3,True,1500.0,0.5
2,2,True,1,True,1500.0,0.5
3,2,True,1,True,1500.0,1.0
4,3,True,2,True,1500.0,1.0


In [47]:
assert(model.mu == 0)
max_iter = 1000

solver_kwargs = {'eps_abs': 100, 'max_iter': max_iter, 'stop_crit': 'max_iter',
                 'verbose': True, 'verbose_step': 500, 'save_history': True}
tic = time.time()
result = model.find_equilibrium(solver_name = 'ustm', composite = True,
                                solver_kwargs = solver_kwargs,
                                base_flows = alpha * graph_data['graph_table']['capacity'])
                                #base_flows here doesn't define anything now
toc = time.time()
print('Elapsed time: {:.0f} sec'.format(toc - tic))
print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')

result['elapsed_time'] = toc - tic

base_flows = result['flows']

Composite optimization...
Oracles created...
Universal similar triangles method...
Primal_init = 3500
Dual_init = -3500
Duality_gap_init = 1137.5

Iterations number: 500
Inner iterations number: 1011
Primal_func_value = 4898.5
Dual_func_value = -4937.46
Duality_gap = -37.0739
Duality_gap / Duality_gap_init = -0.0325925

Iterations number: 1000
Inner iterations number: 2012
Primal_func_value = 4917.91
Dual_func_value = -4937.44
Duality_gap = -19.3581
Duality_gap / Duality_gap_init = -0.0170181

Result: success
Total iters: 1000
Primal_func_value = 4917.91
Dual_func_value = -4937.44
Duality_gap = -19.3581
Duality_gap / Duality_gap_init = -0.0170181
Oracle elapsed time: 1 sec
Elapsed time: 1 sec
Time ratio = 2.000013396133154
Flow excess = 0.011576154988026222



__Базовые потоки, найденные численно, должны быть неотрицательными и соответствовать ограничениям пропускной способности.__

__Во время этих итераций метрика разрыва двойственности не имеет значения.__ На следующем этапе (в модели Стабильной Динамики) разрыв двойственности, основанный на найденных базовых потоках, будет неотрицательным и будет уменьшаться по мере приближения алгоритма к решению задачи.

In [48]:
print("base flows:", result['flows'])
print("base times:", result['times'])

base flows: [ 486.9120629  1513.0879371  1513.07734953 1486.92265047  861.97682564
 1138.02317436]
base times: [1.         0.99994994 0.99998138 1.         1.         1.0000067 ]


In [49]:
#with open(sd_save + 'anaheim_' + 'ustm' + '_base_flows_max_iter_' + str(max_iter) + '_SD.pickle', 'wb') as f:
#    pickle.dump(base_flows, f)

## Step 2: SD Model solution

In [50]:
graph_data['graph_table']['capacity'] = init_capacities
model = md.Model(graph_data, graph_correspondences,
                 total_od_flow, mu = 0)

graph_data['graph_table'].head()

,init_node,init_node_thru,term_node,term_node_thru,capacity,free_flow_time
0,1,True,3,True,2000.0,1.0
1,1,True,3,True,2000.0,0.5
2,2,True,1,True,2000.0,0.5
3,2,True,1,True,2000.0,1.0
4,3,True,2,True,2000.0,1.0


## Метод Подобных Треугольников

In [51]:
assert(model.mu == 0)
max_iter = 1000

solver_kwargs = {'eps_abs': 100,
                 'max_iter': max_iter, 'stop_crit': 'max_iter',
                 'verbose': True, 'verbose_step': 400, 'save_history': True}
tic = time.time()
result = model.find_equilibrium(solver_name = 'ustm', composite = True,
                                solver_kwargs = solver_kwargs, base_flows = base_flows)
toc = time.time()
print('Elapsed time: {:.0f} sec'.format(toc - tic))
print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')
# NOTE: duality gap should be nonnegative here!

Composite optimization...
Oracles created...
Universal similar triangles method...
Primal_init = 3500
Dual_init = -3500
Duality_gap_init = 956.341

Iterations number: 400
Inner iterations number: 811
Primal_func_value = 4217.82
Dual_func_value = -4249.94
Duality_gap = 23.2547
Duality_gap / Duality_gap_init = 0.0243164

Iterations number: 800
Inner iterations number: 1611
Primal_func_value = 4232.76
Dual_func_value = -4249.98
Duality_gap = 15.967
Duality_gap / Duality_gap_init = 0.0166959

Result: success
Total iters: 1000
Primal_func_value = 4236.71
Dual_func_value = -4249.93
Duality_gap = 9.79134
Duality_gap / Duality_gap_init = 0.0102383
Oracle elapsed time: 1 sec
Elapsed time: 1 sec
Time ratio = 2.00002249809661
Flow excess = 0.00843569531706101



In [52]:
print("flows:", result['flows'])
print("times: ", result['times'])

flows: [  -0.         2000.         2013.92522557  986.07477443  487.34645702
 1512.65354298]
times:  [1.         0.5        1.00001125 1.         1.         0.99989975]
